In [ ]:
# Author: Somnath Tagore, Ph.D. Title: Integrated T-cells TCR diffusion
# Script Name: tcells_clustering.R 
# Last Updated: 05/20/2021

# Packages required for this analysis
library(Seurat)
library(destiny)
library(dplyr)
library(ggplot2)
library(cowplot)
library(scater)
library(SingleCellExperiment)
library(gplots)
library(viridis)
library(scales)
#library(hypeR)


# Color palette
colors <- c('#006E82', '#AA0A3C', '#F0E442', '#00A0FA', '#FA5078', '#005AC8', '#CC79A7', '#FAE6BE', '#0072B2', '#A0FA82', '#F0F032', '#0AB45A',
            '#FA7850', '#14D2DC', '#FA78FA')

colPrimvsBMmain <- c('BRAIN_METS'='#FF0000','Epithelial cells'='#FF00FF','PRIMARY'='#0000FF')
colSTKvsNonSTKmain <- c('Epithelial cells'='#FF00FF', 'Non-STK11-mut'='#FFFF00', 'STK11-mut'='#000000')



colPrimvsBMFine <- c('Airway basal'='#D2691E','Airway ciliated'='#7FFFD4','Airway club'='#D2B48C','Airway goblet'='#808000','AT1'='#BA55D3','AT2'='#FFE4B5','AT2 (high MT)'='#E6E6FA','BRAIN_METS'='#FF0000','NE'='#D8BFD8', 'PRIMARY'='#0000FF')
colSTKvsNonSTKFine <- c('Airway basal'='#D2691E','Airway ciliated'='#7FFFD4','Airway club'='#D2B48C','Airway goblet'='#808000','AT1'='#BA55D3','AT2'='#FFE4B5','AT2 (high MT)'='#E6E6FA', 'NE'='#D8BFD8' ,'Non-STK11-mut'='#FFFF00','STK11-mut'='#000000')


#colPrimvsBMvsSTKvsNonSTKfine <- c('STK11-mut (PRIMARY)'='#808080','STK11-mut (BRAIN_METS)'='#FFA500','Non-STK11-mut (PRIMARY)'='#800080','Non-STK11-mut (BRAIN_METS)'='#008000','Airway basal'='#D2691E','Airway ciliated'='#7FFFD4','Airway club'='#D2B48C','Airway goblet'='#808000','AT1'='#BA55D3','AT2'='#FFE4B5','AT2 (high MT)'='#E6E6FA','NE'='#D8BFD8')
#colPrimvsBMvsSTKvsNonSTKmain <- c('STK11-mut (PRIMARY)'='#808080','STK11-mut (BRAIN_METS)'='#FFA500','Non-STK11-mut (PRIMARY)'='#800080','Non-STK11-mut (BRAIN_METS)'='#008000','Epithelial cells'='#FF00FF')
colPrimvsBMvsSTKvsNonSTKfine <- c('Airway basal'='#D2691E','Airway ciliated'='#7FFFD4','Airway club'='#D2B48C','Airway goblet'='#808000','AT1'='#BA55D3','AT2'='#FFE4B5','AT2 (high MT)'='#E6E6FA','NE'='#D8BFD8','Non-STK11-mut (BRAIN_METS)'='#008000','Non-STK11-mut (PRIMARY)'='#800080','STK11-mut (BRAIN_METS)'='#FFA500','STK11-mut (PRIMARY)'='#808080')
colPrimvsBMvsSTKvsNonSTKmain <- c('Epithelial cells'='#FF00FF','Non-STK11-mut (BRAIN_METS)'='#008000','Non-STK11-mut (PRIMARY)'='#800080','STK11-mut (BRAIN_METS)'='#FFA500','STK11-mut (PRIMARY)'='#808080')





# Read in file
patient <- readRDS('T_cells_v7.rds')

colnames(patient@meta.data)

# Subset to .... cells

table(patient@meta.data$manual.annot)

patient <- subset(patient, manual.annot %in% c('CD8+','Naive','Differentiating T-cells','Tissue-residency, Effector memory, CD4+','Tissue resident memory cells'))

####Cell cycle scoring

s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes

# Create our Seurat object and complete the initalization steps

patient <- FindVariableFeatures(patient, selection.method = "vst")
patient <- ScaleData(patient, features = rownames(patient))

patient <- RunPCA(patient)

patient <- CellCycleScoring(patient, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)

head(patient[[]])

pdf('Group_1_T_cells_res_0.1_refined_upd.pdf')
RidgePlot(patient, features = c("PCNA", "TOP2A", "MCM6", "MKI67"), ncol = 2)

patient <- RunPCA(patient, features = c(s.genes, g2m.genes))
DimPlot(patient)

patient <- ScaleData(patient, vars.to.regress = c("S.Score", "G2M.Score"), features = rownames(patient))

patient <- RunPCA(patient, features = VariableFeatures(patient), nfeatures.print = 10)

patient <- RunPCA(patient, features = c(s.genes, g2m.genes))
DimPlot(patient)

dev.off()


#####

dim(patient@meta.data)

colnames(patient@meta.data)

# Subset to .... cells

table(patient@meta.data$cloneType)


DefaultAssay(patient) <- 'RNA'

# Rerun Seurat workflow
#patient <- ScaleData(object = patient)
#patient <- RunPCA(object = patient)
patient <- NormalizeData(patient)
patient <- FindVariableFeatures(patient, selection.method = "vst")
patient <- ScaleData(patient, features = rownames(patient))

patient <- RunPCA(patient)

patient <- RunUMAP(object = patient, dims = 1:30)

# Diffusion component analysis
es <- as.ExpressionSet(as.data.frame(t(patient@assays$RNA@data)))

es@phenoData@data <- patient@meta.data

# Make diffusion map
dm <- DiffusionMap(es, verbose = T, n_pcs = 30)
saveRDS(dm,file="Group_1_T_cells_res_0.1_refined_upd_dm.rds")

# Add DCs to Seurat object
patient <- AddMetaData(patient, dm@eigenvectors, col.name = colnames(dm@eigenvectors))

pdf('./Group_1_T_cells_res_0.1_refined_upd.pdf')

# Diffusion maps
par(mar = c(5.1, 4.1, 4.1, 2.1), xpd = TRUE)
#palette(c('#0072B2', '#0AB45A'))
plot(dm, col = as.factor(es@phenoData@data$cloneType), main = 'CloneType', pch = 16)
#legend('bottomright', inset = c(0, 0), legend = levels(as.factor(es@phenoData@data$celltype_bped_main)), pch = 16, col = as.factor(levels(as.factor(es@phenoData@data$celltype_bped_main))),
legend('bottomright', inset = c(0, 0), legend = levels(as.factor(es@phenoData@data$cloneType)), pch = 16, col = as.factor(levels(as.factor(es@phenoData@data$cloneType))),
       bty = 'n')
plot(dm, col = as.factor(es@phenoData@data$STK11mut_vs_NonSTK11mut), main = 'STK11mut_vs_NonSTK11mut', pch = 16)
#legend('bottomright', inset = c(0, 0), legend = levels(as.factor(es@phenoData@data$celltype_bped_main)), pch = 16, col = as.factor(levels(as.factor(es@phenoData@data$celltype_bped_main))),
legend('bottomright', inset = c(0, 0), legend = levels(as.factor(es@phenoData@data$STK11mut_vs_NonSTK11mut)), pch = 16, col = as.factor(levels(as.factor(es@phenoData@data$STK11mut_vs_NonSTK11mut))),
       bty = 'n')

plot(dm, col = as.factor(es@phenoData@data$Primary_vs_BM), main = 'Primary_vs_BM', pch = 16)
#legend('bottomright', inset = c(0, 0), legend = levels(as.factor(es@phenoData@data$celltype_bped_main)), pch = 16, col = as.factor(levels(as.factor(es@phenoData@data$celltype_bped_main))),
legend('bottomright', inset = c(0, 0), legend = levels(as.factor(es@phenoData@data$Primary_vs_BM)), pch = 16, col = as.factor(levels(as.factor(es@phenoData@data$Primary_vs_BM))),
       bty = 'n')


dev.off()


dm_df <- as.data.frame(dm@eigenvectors)[sample(nrow(as.data.frame(dm@eigenvectors))),1:3]

pdf('Group_1_T_cells_res_0.1_refined_upd_2Dversion.pdf')

# Diffusion maps
par(mar = c(5.1, 4.1, 4.1, 2.1), xpd = TRUE)

#palette(colPrimvsBMmain)

ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'cloneType']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('cloneType')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'manual.annot']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('manual.annot')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'seurat_clusters']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('seurat_clusters')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'celltype_bped_main']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('celltype_bped_main')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'celltype_bped_fine']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('celltype_bped_fine')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'nCount_RNA']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('nCount_RNA')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'nFeature_RNA']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('nFeature_RNA')+
  theme(legend.title = element_blank())


palette(colPrimvsBMmain)
ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'Primary_vs_BM']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('Primary_vs_BM')+
  theme(legend.title = element_blank())


palette(colSTKvsNonSTKmain)
ggplot(dm_df, aes(DC1, DC2, col = patient@meta.data[rownames(dm_df), 'STK11mut_vs_NonSTK11mut']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('STK11mut_vs_NonSTK11mut')+
  theme(legend.title = element_blank())


dev.off()





Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, 

[1] "orig.ident"                          
 [2] "nCount_RNA"                          
 [3] "nFeature_RNA"                        
 [4] "percent.mt"                          
 [5] "percent.rps"                         
 [6] "percent.rpl"                         
 [7] "percent.rp"                          
 [8] "patient_long"                        
 [9] "patient"                             
[10] "predicted_doublets"                  
[11] "doublet_scores"                      
[12] "nCount_SCT"                          
[13] "nFeature_SCT"                        
[14] "SCT_snn_res.0.8"                     
[15] "seurat_clusters"                     
[16] "celltype_bped_main"                  
[17] "celltype_bped_fine"                  
[18] "celltype_iced_main"                  
[19] "celltype_iced_fine"                  
[20] "celltype_hpca_main"                  
[21] "celltype_hpca_fine"                  
[22] "celltype_mid_main"                   
[23] "celltype_mid_fine"                   
[24] "integrated_snn_res.0.5"              
[25] "manual.annot"                        
[26] "integrated_snn_res.0.2"              
[27] "prim_vs_bm"                          
[28] "stk_vs_nonstk"                       
[29] "Primary_vs_BM"                       
[30] "STK11mut_vs_NonSTK11mut"             
[31] "Primary_BM_STK_NonSTK"               
[32] "Primary_vs_BM_vs_CHEST_WALL_MET"     
[33] "Primary_BM_STK_NonSTK_CHEST_WALL_MET"
[34] "manual.annot.fine"                   
[35] "barcode"                             
[36] "CTgene"                              
[37] "CTnt"                                
[38] "CTaa"                                
[39] "CTstrict"                            
[40] "Frequency"                           
[41] "cloneType"                           
[42] "T_cell_groups"


                                   CD8+                                   Naive 
                                   9122                                    7537 
                Differentiating T-cells                               Ribosomal 
                                   4916                                    2872 
                                  Tregs                               NKT cells 
                                   2572                                     891 
                         T helper cells            Tissue resident memory cells 
                                    592                                      95 
Tissue-residency, Effector memory, CD4+ 
                                     49 

Warning message in FindVariableFeatures.Assay(object = assay.data, selection.method = selection.method, :
“selection.method set to 'vst' but count slot is empty; will use data slot instead”
Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in hvf.info$variance.expected[not.const] <- 10^fit$fitted:
“number of items to replace is not a multiple of replacement length”
Centering and scaling data matrix

PC_ 1 
Positive:  PTPRC, ARHGAP15, THEMIS, FYN, SKAP1, CD96, IKZF1, CHST11, ITK, RIPOR2 
	   BCL11B, FYB1, CAMK4, LINC00861, DOCK8, LINC01934, DOCK10, INPP4B, B2M, ADAMTS16 
	   LCP2, LINC02015, AOAH, ETS1, ITGA4, CD247, ELMO1, IL7R, AC019117.1, APBB1IP 
Negative:  CEACAM5, LCN2, CAV1, S100P, CD36, IGKV1-5, IGHV4-59, CEACAM6, GSTP1, IGHG4 
	   FTH1, CCDC80, SPP1, KRT19, EREG, ALDH1A1, KRT8, MDK, FTL, HSPB1 
	   AGR2, AC010737.1, MAP1B, CST3, AC092957.1, SLC7A11, LINC02163, AC024230.1, AC012494.1, COX6A1 
PC_ 2 
Positive:  IGHV4-59, IGHG4, IGKC, IGKV1-5, IGLV3-1, 

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rps,percent.rpl,percent.rp,patient_long,patient,predicted_doublets,⋯,CTnt,CTaa,CTstrict,Frequency,cloneType,T_cell_groups,S.Score,G2M.Score,Phase,old.ident
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,⋯,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<fct>
KRAS_10_AAACCTGAGACGCTTT-1,KRAS_10,2214,1412,0.13550136,0.1806685,0.00000000,0.1806685,NSCLC_KRAS_10,NSCLC_KRAS_10,FALSE,⋯,NA,NA,NA,NA,NA,Naive/stem-like,0.008565959,-0.009106636,S,1
KRAS_10_AAACCTGGTCATTAGC-1,KRAS_10,1799,1176,5.50305725,0.4446915,0.22234575,0.6670372,NSCLC_KRAS_10,NSCLC_KRAS_10,FALSE,⋯,NA,NA,NA,NA,NA,Naive/stem-like,-0.002887869,-0.028295210,G1,1
KRAS_10_AAAGCAACATCACCCT-1,KRAS_10,1559,1057,0.06414368,0.1282874,0.06414368,0.1924310,NSCLC_KRAS_10,NSCLC_KRAS_10,FALSE,⋯,NA,NA,NA,NA,NA,Naive/stem-like,0.207857377,0.050922231,S,1
KRAS_10_AAAGTAGTCAGGTAAA-1,KRAS_10,1856,1200,2.85560345,0.2693966,0.10775862,0.3771552,NSCLC_KRAS_10,NSCLC_KRAS_10,FALSE,⋯,NA,NA,NA,NA,NA,CD8 Activated/EM,0.019450081,-0.005371077,S,0
KRAS_10_AAATGCCAGGTTCCTA-1,KRAS_10,1062,784,2.73069680,0.4708098,0.28248588,0.7532957,NSCLC_KRAS_10,NSCLC_KRAS_10,FALSE,⋯,NA,NA,NA,NA,NA,Naive/stem-like,-0.014070173,0.015060721,G2M,1
KRAS_10_AAATGCCTCTGACCTC-1,KRAS_10,1447,1038,0.13821700,0.4146510,0.00000000,0.4146510,NSCLC_KRAS_10,NSCLC_KRAS_10,FALSE,⋯,NA,NA,NA,NA,NA,CD8 Activated/EM,0.030098257,0.010993432,S,0


...

png 
  2

[1] 21719    46

[1] "orig.ident"                          
 [2] "nCount_RNA"                          
 [3] "nFeature_RNA"                        
 [4] "percent.mt"                          
 [5] "percent.rps"                         
 [6] "percent.rpl"                         
 [7] "percent.rp"                          
 [8] "patient_long"                        
 [9] "patient"                             
[10] "predicted_doublets"                  
[11] "doublet_scores"                      
[12] "nCount_SCT"                          
[13] "nFeature_SCT"                        
[14] "SCT_snn_res.0.8"                     
[15] "seurat_clusters"                     
[16] "celltype_bped_main"                  
[17] "celltype_bped_fine"                  
[18] "celltype_iced_main"                  
[19] "celltype_iced_fine"                  
[20] "celltype_hpca_main"                  
[21] "celltype_hpca_fine"                  
[22] "celltype_mid_main"                   
[23] "celltype_mid_fine"                   
[24] "integrated_snn_res.0.5"              
[25] "manual.annot"                        
[26] "integrated_snn_res.0.2"              
[27] "prim_vs_bm"                          
[28] "stk_vs_nonstk"                       
[29] "Primary_vs_BM"                       
[30] "STK11mut_vs_NonSTK11mut"             
[31] "Primary_BM_STK_NonSTK"               
[32] "Primary_vs_BM_vs_CHEST_WALL_MET"     
[33] "Primary_BM_STK_NonSTK_CHEST_WALL_MET"
[34] "manual.annot.fine"                   
[35] "barcode"                             
[36] "CTgene"                              
[37] "CTnt"                                
[38] "CTaa"                                
[39] "CTstrict"                            
[40] "Frequency"                           
[41] "cloneType"                           
[42] "T_cell_groups"                       
[43] "S.Score"                             
[44] "G2M.Score"                           
[45] "Phase"                               
[46] "old.ident"


Hyperexpanded (0.1 < X <= 1)      Large (0.01 < X <= 0.1) 
                          40                          590 
  Medium (0.001 < X <= 0.01) 
                         662 

Centering and scaling data matrix

PC_ 1 
Positive:  SDK1, MACROD2, NAALADL2, DANT2, DNAH14, ZNF117, FGF13, NFIB, KCNQ1OT1, PPP1R9A 
	   DOCK1, CADPS2, PTK2, MUC16, TTC6, FARP1, RIMS2, LIMCH1, EYA2, MIPOL1 
	   ABCC3, AL008633.1, RANBP17, ARHGEF38, CNTNAP2, PSD3, MTUS1, DST, DNAH11, GPR137C 
Negative:  AOAH, B2M, TOX, PDE3B, CNOT6L, CCL5, IL7R, DTHD1, CDC14A, LINC01619 
	   XIST, BCL2, ITGA1, RUNX2, TTN, CD69, NELL2, EPSTI1, CRYBG1, RBPJ 
	   CD38, MGAT5, STAT1, ITGAE, TRPS1, TNFAIP3, PPP2R2B, LYST, KLRD1, NKG7 
PC_ 2 
Positive:  MAP1B, SYT1, CLU, SPARCL1, PCSK1N, SNAP25, RTN1, GPM6A, YWHAH, UCHL1 
	   HSP90AB1, ANK2, THY1, GAPDH, CPE, MAP1A, CAMK2A, PEBP1, MT3, GRIN2B 
	   FTH1, CREG2, GRIA2, NEFL, OLFM1, CKB, ATP6V0C, KIF1A, TMEM59L, CST3 
Negative:  SDK1, ZNF117, DNAH14, MUC16, DANT2, KCNQ1OT1, AC092535.4, TTC6, EYA2, MACROD2 
	   FGF13, NAALADL2, DNAH11, ARHGEF38, ABCC3, LINC00342, AP001636.3, MIPOL1, GRHL2, GNA14 
	   TMC5, RANBP17, ANKRD36C, LINC00271, SNED1, LINC01091, AL008633.

In [1]:
# Author: Somnath Tagore, Ph.D. Title: Integrated T-cells TCR diffusion
# Script Name: tcells_clustering.R 
# Last Updated: 05/20/2021

# Packages required for this analysis
library(Seurat)
library(destiny)
library(dplyr)
library(ggplot2)
library(cowplot)
library(scater)
library(SingleCellExperiment)
library(gplots)
library(viridis)
library(scales)
# Read in file


Attaching SeuratObject

Attaching sp


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, c

In [6]:
patient <- readRDS('T_cells_v7.rds')


In [27]:
DefaultAssay(patient) <- 'RNA'

tcf7_expression <- FetchData(patient, vars = "TCF7")
tox_expression <- FetchData(patient, vars = "TOX")

# Add the expression data to the metadata of the Seurat object
patient[["TCF7_expression"]] <- tcf7_expression
patient[["TOX_expression"]] <- tox_expression


In [2]:
meta_data <- readRDS("Group_1_T_cells_CD8_dm.rds")


In [29]:
group1 <- "Primary_vs_BM"
group2 <- "STK11mut_vs_NonSTK11mut"
# Ensure the genes are present in the Seurat object
# Align cells in Seurat object and diffusion map
common_cells <- intersect(Cells(patient), rownames(meta_data@eigenvectors))
seurat_obj <- subset(patient, cells = common_cells)
meta_data_eigenvectors <- meta_data@eigenvectors[common_cells, ]
# Subset the data by groups


In [30]:

write.csv(meta_data_eigenvectors[,1:3], "dm.csv")
selected_columns <- seurat_obj@meta.data[, c('Primary_vs_BM', 'STK11mut_vs_NonSTK11mut', 'cloneType', 'TCF7_expression', 'TOX_expression')]
write.csv(selected_columns, "meta_data.csv")



In [10]:
unique(seurat_obj@meta.data$cloneType)

[1] NA                             "Large (0.01 < X <= 0.1)"     
[3] "Medium (0.001 < X <= 0.01)"   "Hyperexpanded (0.1 < X <= 1)"

In [4]:
dm <- readRDS("Group_1_T_cells_CD8_dm.rds")


In [5]:
colors <- c('#006E82', '#AA0A3C', '#F0E442', '#00A0FA', '#FA5078', '#005AC8', '#CC79A7', '#FAE6BE', '#0072B2', '#A0FA82', '#F0F032', '#0AB45A',
            '#FA7850', '#14D2DC', '#FA78FA')

colPrimvsBMmain <- c('BRAIN_METS'='#FF0000','Epithelial cells'='#FF00FF','PRIMARY'='#0000FF')
colSTKvsNonSTKmain <- c('Epithelial cells'='#FF00FF', 'Non-STK11-mut'='#FFFF00', 'STK11-mut'='#000000')



colPrimvsBMFine <- c('Airway basal'='#D2691E','Airway ciliated'='#7FFFD4','Airway club'='#D2B48C','Airway goblet'='#808000','AT1'='#BA55D3','AT2'='#FFE4B5','AT2 (high MT)'='#E6E6FA','BRAIN_METS'='#FF0000','NE'='#D8BFD8', 'PRIMARY'='#0000FF')
colSTKvsNonSTKFine <- c('Airway basal'='#D2691E','Airway ciliated'='#7FFFD4','Airway club'='#D2B48C','Airway goblet'='#808000','AT1'='#BA55D3','AT2'='#FFE4B5','AT2 (high MT)'='#E6E6FA', 'NE'='#D8BFD8' ,'Non-STK11-mut'='#FFFF00','STK11-mut'='#000000')


#colPrimvsBMvsSTKvsNonSTKfine <- c('STK11-mut (PRIMARY)'='#808080','STK11-mut (BRAIN_METS)'='#FFA500','Non-STK11-mut (PRIMARY)'='#800080','Non-STK11-mut (BRAIN_METS)'='#008000','Airway basal'='#D2691E','Airway ciliated'='#7FFFD4','Airway club'='#D2B48C','Airway goblet'='#808000','AT1'='#BA55D3','AT2'='#FFE4B5','AT2 (high MT)'='#E6E6FA','NE'='#D8BFD8')
#colPrimvsBMvsSTKvsNonSTKmain <- c('STK11-mut (PRIMARY)'='#808080','STK11-mut (BRAIN_METS)'='#FFA500','Non-STK11-mut (PRIMARY)'='#800080','Non-STK11-mut (BRAIN_METS)'='#008000','Epithelial cells'='#FF00FF')
colPrimvsBMvsSTKvsNonSTKfine <- c('Airway basal'='#D2691E','Airway ciliated'='#7FFFD4','Airway club'='#D2B48C','Airway goblet'='#808000','AT1'='#BA55D3','AT2'='#FFE4B5','AT2 (high MT)'='#E6E6FA','NE'='#D8BFD8','Non-STK11-mut (BRAIN_METS)'='#008000','Non-STK11-mut (PRIMARY)'='#800080','STK11-mut (BRAIN_METS)'='#FFA500','STK11-mut (PRIMARY)'='#808080')
colPrimvsBMvsSTKvsNonSTKmain <- c('Epithelial cells'='#FF00FF','Non-STK11-mut (BRAIN_METS)'='#008000','Non-STK11-mut (PRIMARY)'='#800080','STK11-mut (BRAIN_METS)'='#FFA500','STK11-mut (PRIMARY)'='#808080')


In [6]:

ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'cloneType']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('cloneType')+
  theme(legend.title = element_blank())

In [ ]:
pdf('Group_1_T_cells_res_0.1_refined_upd_2Dversion.pdf')

# Diffusion maps
par(mar = c(5.1, 4.1, 4.1, 2.1), xpd = TRUE)

#palette(colPrimvsBMmain)

ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'cloneType']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('cloneType')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'manual.annot']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('manual.annot')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'seurat_clusters']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('seurat_clusters')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'celltype_bped_main']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('celltype_bped_main')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'celltype_bped_fine']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('celltype_bped_fine')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'nCount_RNA']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('nCount_RNA')+
  theme(legend.title = element_blank())

ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'nFeature_RNA']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('nFeature_RNA')+
  theme(legend.title = element_blank())


palette(colPrimvsBMmain)
ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'Primary_vs_BM']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('Primary_vs_BM')+
  theme(legend.title = element_blank())


palette(colSTKvsNonSTKmain)
ggplot(dm_df, aes(DC1, DC2, col = meta_data[rownames(dm_df), 'STK11mut_vs_NonSTK11mut']))+
  geom_point(alpha = 1,shape = 16,size = 1.5) + theme_classic() + ggtitle('STK11mut_vs_NonSTK11mut')+
  theme(legend.title = element_blank())


dev.off()



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

